In [1]:
import numpy as np
import pandas as pd
# import nltk
# import re

# import gspread
# from gspread.exceptions import WorksheetNotFound
# from gspread_dataframe import get_as_dataframe, set_with_dataframe


from Levenshtein import ratio
from fuzzy import Soundex
# import scipy.sparse as sps
# import scipy.cluster.hierarchy as sch

from collections import Counter

# import spacy
# from spacy.lemmatizer import Lemmatizer
# from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Whisker, Plot
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet, FactorRange
from bokeh.models.graphs import from_networkx

from bokeh.layouts import gridplot



# from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, \
# BoxZoomTool, WheelZoomTool, ResetTool, PanTool, ColumnDataSource, LabelSet

# from bokeh.plotting import figure
# from bokeh.transform import factor_cmap
from bokeh.palettes import Category20_20, Accent

# from bokeh.layouts import row

# from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx

# from bokeh.models.graphs import from_networkx

import random
import itertools
import math

import pprint
pp = pprint.PrettyPrinter(indent=4)




# from iteration_utilities import flatten

# import matplotlib as mpl
# import matplotlib.pyplot as plt

In [2]:
depositions_df = pd.read_json('../converted_json/all_depositions.json')


In [285]:
depositions_df.iloc[4186]

creation_date                                                   1643-3-10
creation_place                                     {'country': 'Ireland'}
deposition_type                                                   [Bysse]
filename                                           dep_824148r131_tei.xml
nature_of_deposition                   [arson, Multiple Killing, Robbery]
participants_number                                                    11
people_list             [{'role': 'Deponent', 'forename': 'William', '...
signed_by               [{'role': 'Commissioner', 'name': 'Philip Byss...
title                                                                 NaN
creation_date_period                                           1643-03-10
creation_year                                                        1643
Name: 4186, dtype: object

In [4]:
depositions_df['creation_date_period'] = pd.PeriodIndex(depositions_df['creation_date'], freq='d')

In [5]:
depositions_df.dtypes

creation_date           object
creation_place          object
deposition_type         object
filename                object
nature_of_deposition    object
participants_number      int64
people_list             object
signed_by               object
title                   object
creation_date_period    object
dtype: object

In [6]:
depositions_df['creation_year'] = depositions_df['creation_date'].str.slice(start=0, stop=4)

In [7]:
depositions_df['creation_year'] = depositions_df['creation_year'].fillna('Unknown')

In [8]:
group_a = depositions_df.groupby('creation_year')

In [9]:
group_a.describe()

participants_number                                           \
                            count       mean        std   min    25%   50%   
creation_year                                                                
1641                        145.0   5.496552   4.725998   0.0   3.00   4.0   
1642                       3124.0   8.501601  13.694543   0.0   2.00   5.0   
1643                        742.0   9.548518  17.634226   0.0   1.00   5.0   
1644                        231.0   8.614719  13.316010   0.0   0.00   4.0   
1645                         98.0  14.551020  52.305826   0.0   1.00   5.0   
1646                         50.0   8.520000  17.888818   0.0   1.00   3.0   
1647                         15.0   6.600000   6.957011   1.0   3.50   5.0   
1650                          2.0   6.000000   5.656854   2.0   4.00   6.0   
1652                        295.0   8.962712   7.276042   0.0   4.00   7.0   
1653                        972.0   7.471193   6.629071   0.0   3.00   6.0   
1654                        991.0   4.063572   3.817159   0.0   2.00   3.0   
1655                          4.0   5.500000   3.696846   2.0   2.75   5.0   
1656                          1.0  33.000000        NaN  33.0  33.00  33.0   
1660                          2.0   3.000000   0.000000   3.0   3.00   3.0   
1662                          4.0   8.000000   6.271629   3.0   3.00   6.5   
1667                          1.0   3.000000        NaN   3.0   3.00   3.0   
Unknown                     334.0   7.802395  16.810457   0.0   1.00   5.0   

                             
                 75%    max  
creation_year                
1641            7.00   39.0  
1642           10.00  210.0  
1643           11.00  253.0  
1644           10.00  101.0  
1645           12.00  488.0  
1646            7.00  100.0  
1647            7.00   29.0  
1650            8.00   10.0  
1652           11.00   55.0  
1653            9.00   66.0  
1654            5.00   35.0  
1655            7.75   10.0  
1656           33.00   33.0  
1660            3.00    3.0  
1662           11.50   16.0  
1667            3.00    3.0  
Unknown         9.00  261.0

# Number of depositions per year

In [10]:
output_notebook()

p = figure(plot_width=800, plot_height=600, x_range=group_a, title="Depositions by Year",
           toolbar_location=None, tools="")

p.vbar(x='creation_year', top='participants_number_count', width=0.8, source=group_a)

p.y_range.start = 0
p.xgrid.grid_line_color = None

show(p)

Loading BokehJS ...

# Number of participants per deposition

In [11]:
depositions_df['participants_number'].describe()

count    7011.000000
mean        7.848952
std        13.790283
min         0.000000
25%         2.000000
50%         5.000000
75%         9.000000
max       488.000000
Name: participants_number, dtype: float64

In [12]:
depositions_df[depositions_df['participants_number'] == 0]

,creation_date,creation_place,deposition_type,filename,nature_of_deposition,participants_number,people_list,signed_by,title,creation_date_period,creation_year
0,NaN,{'country': 'Ireland'},NaN,dep_809000r000_tei.xml,[Titlepage],0,[],NaN,NaN,NaT,Unknown
88,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809166r088_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12,1641
89,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809168r089_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12,1641
90,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809170r090_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12,1641
92,1641-11-12,{'country': 'Ireland'},[Other Copy],dep_809174r092_tei.xml,"[words, Investigative]",0,[],NaN,NaN,1641-11-12,1641
232,NaN,{'country': 'Ireland'},NaN,dep_810000r000a_tei.xml,[Title page],0,[],NaN,NaN,NaT,Unknown
233,NaN,{'country': 'Ireland'},NaN,dep_810001r001_tei.xml,[Index],0,[],NaN,NaN,NaT,Unknown
234,1642-2-5,{'country': 'Ireland'},[Waring Copy],dep_810003r002_tei.xml,NaN,0,[],NaN,NaN,1642-02-05,1642
235,1642-2-26,{'country': 'Ireland'},[Waring Copy],dep_810003v003_tei.xml,NaN,0,[],NaN,NaN,1642-02-26,1642
236,1642-2-28,{'country': 'Ireland'},[Waring Copy],dep_810004r004_tei.xml,NaN,0,[],NaN,NaN,1642-02-28,1642


In [13]:
part_hist, edges = np.histogram(depositions_df['participants_number'], bins = list(range(0,50,1)))
participants = pd.DataFrame({'part_hist' : part_hist, 'left': edges[:-1], 'right': edges[1:]})   

In [14]:
participants

,part_hist,left,right
0,998,0,1
1,347,1,2
2,918,2,3
3,637,3,4
4,601,4,5
5,555,5,6
6,431,6,7
7,342,7,8
8,293,8,9
9,240,9,10


In [15]:
output_notebook()

p = figure(plot_width=800, 
           plot_height=600, 
           title="Histogram of Participants (up to 100) in 1641 Depositions",
           x_axis_label = "Number of participants",
           y_axis_label = "Number of depositions")

p.quad(bottom=0,
       top=participants['part_hist'],
       left=participants['left'],
       right=participants['right'],
       fill_color='red',
       line_color='black')

show(p)

Loading BokehJS ...

# People's names

In [459]:
name_counter = Counter()
people_list = depositions_df['people_list'].tolist()
names_list = []
person2indx = {}
#All participants have only one role (checked)
indx2role = {}
for i, dep_part in enumerate(people_list):
    names = []
    for person in dep_part:
        person_str = ''
        if 'forename' in person:
            person_str += person['forename']
        else:
            person_str += 'None'
        person_str += ' '
        
        if 'surname' in person:
            person_str += person['surname']
        else:
            person_str += 'None'
    
        if person_str not in person2indx:
            idx = len(person2indx)
            person2indx[person_str] = idx
            indx2role[idx] = {i : person['role']}
            if person_str == 'Dauid Roch':
                print('1',person_str, person['role'], i, idx)
        else:
            idx = person2indx[person_str]
            indx2role[idx][i] = person['role']
            if person_str == 'Dauid Roch':
                print('2',person_str, person['role'], i)
        name_counter[person_str] += 1
        names.append(person_str)
    names_list.append(names)

indx2person = {v:k for k,v in person2indx.items()}

1 Dauid Roch Deponent 783 3938
2 Dauid Roch Debtor 3642
2 Dauid Roch Debtor 3649
2 Dauid Roch Victim 3663
2 Dauid Roch Debtor 3664
2 Dauid Roch Victim 3856
2 Dauid Roch Rebel 3871
2 Dauid Roch Debtor 3969
2 Dauid Roch Rebel 3989
2 Dauid Roch Rebel 4010
2 Dauid Roch Rebel 4186
2 Dauid Roch Rebel 4810


In [460]:
for i,name in enumerate(name_counter.most_common(1000)):
    if name[0].split(' ')[0] == '*':
        print(i, name)

4 ('* Carthy', 59)
12 ('* None', 53)
37 ('* Croaly', 31)
216 ('* Cusack', 12)
281 ('* Eustace', 10)
293 ('* Butler', 10)
333 ('* Hurly', 10)
339 ('* Plunkett', 9)
370 ('* Preston', 9)
423 ('* King', 8)
428 ('* Butlers', 8)
436 ('* Plunckett', 8)
497 ('* Rely', 7)
502 ('* Birne', 7)
549 ('* Smith', 7)
554 ('* Darcie', 7)
620 ('* Coghlane', 7)
647 ('* Browne', 6)
672 ('* Walsh', 6)
707 ('* walsh', 6)
714 ('* Rochford', 6)
725 ('* Jackson', 6)
737 ('* Bath', 6)
745 ('* Baker', 6)
751 ('* Heslenan', 6)
790 ('* Roche', 6)
815 ('* Cullane', 6)
818 ('* Canty', 6)
867 ('* Bermingham', 5)
877 ('* Martin', 5)
890 ('* ffox', 5)
904 ('* Scurlock', 5)
914 ('* Queen', 5)
938 ('* Jones', 5)
945 ('* Brenans', 5)
947 ('* Ormond', 5)
967 ('* Dempsie', 5)
968 ('* Hussey', 5)
971 ('* Allen', 5)
974 ('* Welsh', 5)
982 ('* Parsons', 5)


In [461]:
sorted(list(filter(lambda x: x.split(' ')[0] == '*', name_counter.keys())))

['* Adams',
 '* Alfrey',
 '* Allen',
 '* Apsley',
 '* Archbold',
 '* Archbould',
 '* Ardagh',
 '* Argile',
 '* Ashpole',
 '* Atturney',
 '* Aungiers',
 '* Awdley',
 '* Axtell',
 '* Baale',
 '* Babe',
 '* Bagenall',
 '* Bagnall',
 '* Bagnell',
 '* Bagnoll',
 '* Baker',
 '* Baldwin',
 '* Balife',
 '* Balliden',
 '* Bane',
 '* Banister',
 '* Banks',
 '* Bannister',
 '* Baretts',
 '* Barnard',
 '* Barnwall',
 '* Barnwalls',
 '* Barnwell',
 '* Baron of the Navan',
 '* Barret',
 '* Barrets',
 '* Barretts',
 '* Barry',
 '* Barten',
 '* Barwick',
 '* Bateman',
 '* Bath',
 '* Bathe',
 '* Baxter',
 '* Bayly',
 '* Bealeing',
 '* Beane',
 '* Beatagh',
 '* Bedlow',
 '* Bellew',
 '* Benjamin',
 '* Bennet',
 '* Bennett',
 '* Bermingham',
 '* Berne',
 '* Berrye',
 '* Berte',
 '* Betagh',
 '* Betaghe',
 '* Bewlan',
 '* Bifeild',
 '* Bingham',
 '* Binghams',
 '* Biram',
 '* Birmingham',
 '* Birne',
 '* Birnes',
 '* Birns',
 '* Birt',
 '* Blanie',
 '* Blassell',
 '* Bocannon',
 '* Boile',
 '* Boise',
 '*

In [462]:
sorted(list(filter(lambda x: x.split(' ')[1] == 'Roch', name_counter.keys())))

['* Roch',
 'Adam Roch',
 'Alexander Roch',
 'Alixander Roch',
 'Allexander Roch',
 'Andrew Roch',
 'Ann Roch',
 'Art Roch',
 'Bele Roch',
 'Captain Roch',
 'Dauid Roch',
 'Dauis Roch',
 'David Roch',
 'Davy Roch',
 'Dominick Roch',
 'Edmond Roch',
 'Edmund Roch',
 'Edward Roch',
 'Ellec Roch',
 'Ellick Roch',
 'Ellin Roch',
 'Garratte Roch',
 'Garret Roch',
 'James Roch',
 'James Roch fits patrike',
 'John Roch',
 'Jordan Roch',
 'Lady Roch',
 'Lawrence Roch',
 'Lo: Roch',
 'Lord Roch',
 'Margarett Roch',
 'Margrett Roch',
 'Mathew Roch',
 'Maurice Roch',
 'Mawrish Roch',
 'Morris Roch',
 'Patrick Roch',
 'Patrick Roch ffitz Richard',
 'Patricke Roch',
 'Patrike Roch',
 'Pearce Roch',
 'Peirc Roch',
 'Peirce Roch',
 'Peirse Roch',
 'Philip Roch',
 'Philip Roch ffitz Patrick',
 'Phillick Roch',
 'Phillip Roch',
 'Phillipp Roch',
 'Pierce Roch',
 'Redmond Roch',
 'Richard Roch',
 'Robert Roch',
 'Roberte Roch',
 'Stiuen Roch',
 'Theobald Roch',
 'Tho: Roch',
 'Thomas Roch',
 'Tibbett Ro

# People network

In [463]:
soundex = Soundex(4)

In [540]:
G = nx.Graph()
for i, dep_names in enumerate(names_list):
    for person_a, person_b in itertools.combinations(dep_names, 2):
        G.add_edge(person2indx[person_a], person2indx[person_b], deposition=i)
    

In [541]:
print(names_list[783])
print(people_list[4010])

['Dauid Roch', 'Robert Kennedy', 'Nathaniell Snapp', 'John Leeson', 'Siluester Kennedy', 'Earle of Meath', 'Luke Toole']
[{'role': 'Deponent', 'forename': 'William', 'surname': 'ffield', 'occupation': 'Gardener', 'sex': '1', 'residence': {'placeName': 'Michelstowne', 'region': 'Cork', 'country': 'Ireland'}}, {'role': 'Debtor', 'forename': 'Dauid', 'surname': 'Gibbon', 'sex': '1'}, {'role': 'Rebel', 'forename': 'William', 'surname': 'lord Baron of', 'sex': '1'}, {'role': 'Rebel', 'forename': 'Patricke', 'surname': 'Purcell', 'sex': '3'}, {'role': 'Rebel', 'forename': 'Sir Edward', 'surname': 'fitz Harris', 'sex': '3'}, {'role': 'Rebel', 'forename': 'Thomas', 'surname': 'mc Cragh', 'sex': '3'}, {'role': 'Rebel', 'forename': 'Donogh', 'surname': 'mc Cragh', 'sex': '3'}, {'role': 'Rebel', 'forename': 'Dauid', 'surname': 'Roch', 'sex': '3'}, {'role': 'Rebel', 'forename': 'lord of', 'surname': 'Muskrie', 'sex': '3'}, {'role': 'Victim', 'forename': 'Anselmus', 'surname': 'Adams', 'sex': '1'},

In [542]:
indx2role

{0: {1: 'Deponent',
  380: 'Mentioned',
  793: 'Mentioned',
  794: 'Mentioned',
  797: 'Mentioned',
  817: 'Mentioned',
  818: 'Mentioned',
  940: 'Deponent',
  941: 'Deponent',
  942: 'Deponent',
  943: 'Deponent',
  1399: 'Mentioned',
  1400: 'Mentioned',
  1401: 'Mentioned',
  3082: 'Mentioned',
  3087: 'Mentioned',
  3387: 'Victim',
  4612: 'Victim',
  4655: 'Victim',
  5104: 'Victim',
  5105: 'Victim',
  5109: 'Presenter',
  5110: 'Examiner',
  5585: 'Mentioned',
  5586: 'Commissioner'},
 1: {1: 'Mentioned'},
 2: {1: 'Mentioned'},
 3: {1: 'Mentioned'},
 4: {1: 'Rebel', 2232: 'Rebel', 2322: 'Rebel', 4870: 'Debtor'},
 5: {1: 'Mentioned', 412: 'Mentioned'},
 6: {1: 'Mentioned', 1274: 'Mentioned', 5027: 'Mentioned', 5796: 'Mentioned'},
 7: {1: 'Mentioned', 31: 'Mentioned'},
 8: {1: 'Mentioned'},
 9: {1: 'Mentioned', 1266: 'Mentioned', 1267: 'Mentioned'},
 10: {1: 'Rebel', 1972: 'Rebel', 5585: 'Rebel', 5603: 'Rebel', 5700: 'Rebel'},
 11: {1: 'Rebel'},
 12: {1: 'Rebel',
  20: 'Mentioned

In [543]:
role_counter = Counter([a for d in indx2role.values() for a in d.values()])

role2color = {n[0]:Category20_20[i] for i,n in enumerate(role_counter.most_common(20))}

In [544]:
role2color

{'Rebel': '#1f77b4',
 'Mentioned': '#aec7e8',
 'Victim': '#ff7f0e',
 'Deponent': '#ffbb78',
 'Debtor': '#2ca02c',
 'Confederate': '#98df8a',
 'Denounced': '#d62728',
 'Apostate': '#ff9896',
 'Witness': '#9467bd',
 'Succour': '#c5b0d5',
 'Proxy': '#8c564b',
 'Landlord': '#c49c94',
 'Creditor': '#e377c2',
 'Interpreter': '#f7b6d2',
 'Author': '#7f7f7f',
 'Examiner': '#c7c7c7',
 'Commissioner': '#bcbd22',
 'Defendant': '#dbdb8d',
 'Recipient': '#17becf',
 'Recognizance': '#9edae5'}

In [545]:
for node in G.nodes():
    G.nodes[node]['full_name'] = indx2person[node]

In [546]:
adams_indxs = {k:v for k,v in person2indx.items() if k.split(" ")[1] == "Adams"}


In [547]:
adams_indxs

{'Randall Adams': 1268,
 'Thomas Adams': 1564,
 'Will: Adams': 8434,
 'Randle Adams': 11154,
 'John Adams': 15258,
 'William Adams': 18973,
 'Anselmus Adams': 20276,
 'Ancelmus Adams': 21417,
 '* Adams': 22653,
 'Richard Adams': 23206,
 'Margrett Adams': 23209,
 'Mary Adams': 25300,
 'Ellen Adams': 31901,
 'Anne Adams': 32842,
 'Giles Adams': 33167,
 'Gyles Adams': 33171,
 'Abraham Adams': 35653}

In [548]:
def merge_nodes(G,nodes, new_node, attr_dict=None):
    """
    Merges the selected `nodes` of the graph G into one `new_node`,
    meaning that all the edges that pointed to or from one of these
    `nodes` will point to or from the `new_node`.
    attr_dict and **attr are defined as in `G.add_node`.
    """
    
    G.add_node(new_node, **attr_dict) # Add the 'merged' node
    edges_to_add = []
    for n1,n2,data in G.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in nodes:
            edges_to_add.append((new_node,n2,data))
        elif n2 in nodes:
            edges_to_add.append((n1,new_node,data))
    
    G.add_edges_from(edges_to_add)
    
    for n in nodes: # remove the merged nodes
        G.remove_node(n)

In [549]:
def get_subgraph_for_pair(name_1, name_2, G):
    name_1_indx = person2indx[name_1]
    name_2_indx = person2indx[name_2]
    pair = [name_1_indx, name_2_indx]
    nodes = []
    graph_1 = nx.ego_graph(G, name_1_indx)
    graph_2 = nx.ego_graph(G, name_2_indx)
    merge_graph = nx.compose(graph_1, graph_2)
    
    remove_edges = [] 
    for edge in merge_graph.edges():
        if name_1_indx not in edge and name_2_indx not in edge:
            remove_edges.append(edge)
    merge_graph.remove_edges_from(remove_edges)
    
    depositions = set([edge[2]['deposition'] for edge in merge_graph.edges(data=True)])
    
    for node in merge_graph.nodes(data=True):
        print(node)
        print(indx2role[node[0]].keys())
        print(depositions)
        role_key = list(depositions.intersection(set(indx2role[node[0]].keys())))[0]
        node_role = indx2role[node[0]][role_key]
            
        merge_graph.nodes[node[0]]['role'] = node_role
        merge_graph.nodes[node[0]]['role_color'] = role2color[node_role] if node_role in role2color else "black"
        
        if node[0] == name_1_indx or node[0] == name_2_indx:
            merge_graph.nodes[node[0]]['size'] = 0.07
        else:
            merge_graph.nodes[node[0]]['size'] = 0.035

    
    
    print(len(depositions))
    deposition_colors = {}
    scale_i = 0
    for edge in merge_graph.edges(data=True):
        if edge[2]['deposition'] not in deposition_colors:
            deposition_colors[edge[2]['deposition']] = Accent[3][scale_i]
            scale_i += 1
        edge[2]['deposition_color'] = deposition_colors[edge[2]['deposition']]
        edge[2]['line_alpha'] = 1
        
    node_comb = itertools.combinations(merge_graph.nodes, 2)
#     invisible_edges = []
#     for idx_a, idx_b in node_comb:
#         sim = ratio(indx2person[idx_a], indx2person[idx_b])
#         if sim > 0.8:
#             print(indx2person[idx_a], '/', indx2person[idx_b])
# #             sound_a = " ".join(soundex(k) for k in indx2person[idx_a].split(" "))
# #             sound_b = " ".join(soundex(k) for k in indx2person[idx_b].split(" "))
# #             print(sound_a, sound_b)
# #             print('----------')
#             invisible_edges.append((idx_a, idx_b, {'inv' : True, 
#                                                    'similarity' : sim,
#                                                    'line_alpha': 0,
#                                                    'weight': 5 + sim}))
                                   
#     merge_graph.add_edges_from(invisible_edges)
    
    merge_pairs = {}
    for idx_a, idx_b in node_comb:
        sim = ratio(indx2person[idx_a], indx2person[idx_b])
        if sim > 0.8:
            attrs = {
                'full_name' : merge_graph.nodes[idx_a]['full_name'],
                'role' : merge_graph.nodes[idx_a]['role'],
                'role_color' : merge_graph.nodes[idx_a]['role_color'],
                'size' : merge_graph.nodes[idx_a]['size']
                
            }
            merge_pairs[(idx_a, idx_b)] = attrs
    
    for (idx_a, idx_b), attrs in merge_pairs.items():
        merge_nodes(merge_graph, [idx_a, idx_b], idx_a+idx_b, attrs)
    

    return merge_graph

# def get_subgraph_for_pair(name_1, name_2, G):
#     print('Hello')
#     name_1_indx = person2indx[name_1]
#     name_2_indx = person2indx[name_2]
#     ego_g = nx.ego_graph(G, name_1_indx)
#     return ego_g
    
    

In [550]:
interactions_c_a = Counter()
for adam_name, indx in adams_indxs.items():
    if 'Anselmus' in adam_name:
        for neighbor in G.neighbors(indx):
            interactions_c_a[neighbor] += 1
# interactions_c.most_common()
print(sorted(list(interactions_c_a.keys())))

[1643, 3938, 14673, 16682, 18313, 20011, 20271, 20272, 20273, 20274, 20275]


In [551]:
# interactions_c_b = Counter()
# for adam_name, indx in adams_indxs.items():
#     if 'Ancelmus' in adam_name:
#         for neighbor in G.neighbors(indx):
#             interactions_c_b[neighbor] += 1
# print(sorted(list(interactions_c_b.keys())))

In [552]:
# interactions_int = set(list(interactions_c_a.keys())).intersection(set(list(interactions_c_b.keys())))
# for el in interactions_int:
#     print(indx2person[el])

In [553]:
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams'),
              ('Randle Adams', 'Randall Adams')]
name_pairs = [('Anselmus Adams', 'Ancelmus Adams'), 
              ('Will: Adams', 'William Adams'), 
              ('Giles Adams', 'Gyles Adams')]
subgraphs = [get_subgraph_for_pair(pair[0], pair[1], G) for pair in name_pairs]

(3938, {'full_name': 'Dauid Roch'})
dict_keys([783, 3642, 3649, 3663, 3664, 3856, 3871, 3969, 3989, 4010, 4186, 4810])
{4010, 4186}
(18313, {'full_name': 'Dauid Gibbon'})
dict_keys([3659, 3888, 4010])
{4010, 4186}
(16682, {'full_name': 'Edward Blissitt'})
dict_keys([3411, 4010, 4186])
{4010, 4186}
(20011, {'full_name': 'Patricke Purcell'})
dict_keys([3989, 3994, 4010, 4872, 4878, 4922, 4929, 4932])
{4010, 4186}
(1643, {'full_name': 'William Browne'})
dict_keys([422, 494, 1806, 2528, 2563, 2694, 2715, 2812, 2817, 2818, 2823, 2824, 2934, 3745, 3958, 4010, 4117, 4186, 5108, 5277, 5279, 6135, 6175])
{4010, 4186}
(20271, {'full_name': 'William ffield'})
dict_keys([4010])
{4010, 4186}
(20272, {'full_name': 'William lord Baron of'})
dict_keys([4010])
{4010, 4186}
(20273, {'full_name': 'Sir Edward fitz Harris'})
dict_keys([4010])
{4010, 4186}
(14673, {'full_name': 'Thomas mc Cragh'})
dict_keys([3105, 3117, 3421, 3428, 3702, 3756, 4010, 4139, 4161, 4195, 4805])
{4010, 4186}
(20274, {'full_name'

In [554]:
indx2role[3938]

{783: 'Deponent',
 3642: 'Debtor',
 3649: 'Debtor',
 3663: 'Victim',
 3664: 'Debtor',
 3856: 'Victim',
 3871: 'Rebel',
 3969: 'Debtor',
 3989: 'Rebel',
 4010: 'Rebel',
 4186: 'Rebel',
 4810: 'Rebel'}

In [555]:
subgraphs[0].nodes(data=True)

NodeDataView({3938: {'full_name': 'Dauid Roch', 'role': 'Rebel', 'role_color': '#1f77b4', 'size': 0.035}, 18313: {'full_name': 'Dauid Gibbon', 'role': 'Debtor', 'role_color': '#2ca02c', 'size': 0.035}, 16682: {'full_name': 'Edward Blissitt', 'role': 'Victim', 'role_color': '#ff7f0e', 'size': 0.035}, 1643: {'full_name': 'William Browne', 'role': 'Victim', 'role_color': '#ff7f0e', 'size': 0.035}, 20271: {'full_name': 'William ffield', 'role': 'Deponent', 'role_color': '#ffbb78', 'size': 0.035}, 20272: {'full_name': 'William lord Baron of', 'role': 'Rebel', 'role_color': '#1f77b4', 'size': 0.035}, 20275: {'full_name': 'lord of Muskrie', 'role': 'Rebel', 'role_color': '#1f77b4', 'size': 0.035}, 21412: {'full_name': 'William Dammer', 'role': 'Deponent', 'role_color': '#ffbb78', 'size': 0.035}, 21413: {'full_name': 'William Castle Connell', 'role': 'Rebel', 'role_color': '#1f77b4', 'size': 0.035}, 21416: {'full_name': 'Lo: Viscont Roch', 'role': 'Mentioned', 'role_color': '#aec7e8', 'size': 

In [556]:
indx2role[3938]

{783: 'Deponent',
 3642: 'Debtor',
 3649: 'Debtor',
 3663: 'Victim',
 3664: 'Debtor',
 3856: 'Victim',
 3871: 'Rebel',
 3969: 'Debtor',
 3989: 'Rebel',
 4010: 'Rebel',
 4186: 'Rebel',
 4810: 'Rebel'}

In [557]:
pp.pprint(depositions_df.iloc[4010].people_list)
print('------')
pp.pprint(depositions_df.iloc[4186].people_list)
print('------')
pp.pprint(depositions_df.iloc[4932].people_list)

[   {   'forename': 'William',
        'occupation': 'Gardener',
        'residence': {   'country': 'Ireland',
                         'placeName': 'Michelstowne',
                         'region': 'Cork'},
        'role': 'Deponent',
        'sex': '1',
        'surname': 'ffield'},
    {'forename': 'Dauid', 'role': 'Debtor', 'sex': '1', 'surname': 'Gibbon'},
    {   'forename': 'William',
        'role': 'Rebel',
        'sex': '1',
        'surname': 'lord Baron of'},
    {'forename': 'Patricke', 'role': 'Rebel', 'sex': '3', 'surname': 'Purcell'},
    {   'forename': 'Sir Edward',
        'role': 'Rebel',
        'sex': '3',
        'surname': 'fitz Harris'},
    {'forename': 'Thomas', 'role': 'Rebel', 'sex': '3', 'surname': 'mc Cragh'},
    {'forename': 'Donogh', 'role': 'Rebel', 'sex': '3', 'surname': 'mc Cragh'},
    {'forename': 'Dauid', 'role': 'Rebel', 'sex': '3', 'surname': 'Roch'},
    {'forename': 'lord of', 'role': 'Rebel', 'sex': '3', 'surname': 'Muskrie'},
    {'foren

In [558]:
output_notebook()
plots = []

for i, G in enumerate(subgraphs[:1]):
    
    #Show network
    plot = Plot(plot_width=800, plot_height=800, x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    plot.title.text = "%s vs %s" % name_pairs[i]
    

  # graph_renderer = from_networkx(G, nx.spring_layout, scale=1, k=0.09, center=(0, 0))
  # graph_renderer = from_networkx(G, nx.kamada_kawai_layout, scale=1.0, dim=2)
    graph_renderer = from_networkx(G, nx.spring_layout, k=.19, center=(0, 0))

    graph_renderer.node_renderer.glyph = Circle(radius="size", fill_color="role_color")
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="deposition_color", 
                                                   line_alpha="line_alpha",
                                                   line_width=2)
    
    node_hover_tool = HoverTool(tooltips=[("full_name", "@full_name"), ("role", "@role")])
#     edge_hover_tool = HoverTool(tooltips=[("deposition", "@deposition")])
#     plot.add_tools(node_hover_tool, edge_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())
    
    plot.renderers.append(graph_renderer)


    x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
    node_labels = [node[1]['full_name'] for node in G.nodes(data=True)]
    
    source = ColumnDataSource({'x': x, 'y': y,
                             'full_name': node_labels})


    labels = LabelSet(x='x', 
                    y='y', 
                    text='full_name', 
                    source=source,
                    background_fill_color='white',
                    background_fill_alpha=0.8,
                    text_align='center', 
                    text_baseline='middle',
                    x_offset=5,
                    y_offset=15,
                    text_font_size='0.6em')

    plot.renderers.append(labels)
    plots.append(plot)

grid = gridplot(plots, ncols=3)
show(grid)


Loading BokehJS ...

# Toponyms

In [36]:
toponyms_list = depositions_df['creation_place'].tolist()
toponyms_counter = Counter()
for i,d in enumerate(toponyms_list):
    for k,v in d.items():
        toponyms_counter[k+':'+v] += 1
print(toponyms_counter.keys())

dict_keys(['country:Ireland'])
